In [1]:
import pandas as pd
import time
import json
import requests
from bs4 import BeautifulSoup
import random
from tqdm import trange, notebook

In [2]:
app_id_1=pd.read_csv("./data/game_name_1.csv", encoding= 'unicode_escape')
app_id_2=pd.read_csv("./data/game_name_2.csv", encoding= 'unicode_escape')
app_id_3=pd.read_csv("./data/game_name_3.csv", encoding= 'unicode_escape')

In [3]:
app_id_1.head()

,appid,name
0,10,Counter-Strike
1,20,Team Fortress Classic
2,40,Deathmatch Classic
3,50,Half-Life: Opposing Force
4,60,Ricochet


In [4]:
# 3일간 추출한 데이터 합치기
app_id_sum=pd.concat([app_id_1,app_id_2,app_id_3])

In [5]:
app_id_sum.drop_duplicates(["appid"], inplace=True)
app_id_sum.drop_duplicates(["name"], inplace=True)

In [6]:
app_id_sum.reset_index(drop=True, inplace=True)

In [7]:
app_id_sum=app_id_sum.set_index('appid')
app_id_sum

,name
appid,
10,Counter-Strike
20,Team Fortress Classic
40,Deathmatch Classic
50,Half-Life: Opposing Force
60,Ricochet
...,...
1335200,Action Taimanin
1340530,Mr. Prepper: Prologue
1362420,Siege Survival: Gloria Victis Prologue


In [8]:
app_id_lst=list(app_id_sum.index)

dic_frame={}
for appid in notebook.tqdm(try_set):
    url=('http://store.steampowered.com/api/appdetails?appids=%s') % (appid)
    try:
        r=requests.get(url)
        result=r.json()
        time.sleep(random.uniform(2,4))
    except:
        pass
    dic_frame.update(result)

In [9]:
dic_frame={}
for appid in notebook.tqdm(app_id_lst):
    url=('http://store.steampowered.com/api/appdetails?appids=%s') % (appid)
    for i in range(2):
        time.sleep(random.uniform(2,4))
        try:
            r=requests.get(url)
            result=r.json()
            break
        except:
            pass
    dic_frame.update(result)

In [10]:
def extract_game_info(dic_frame,appid_lst,name_list,description_list,price,categor,genr,recom):
    for  num in appid_lst:
        temp_category_list=[]
        temp_len=0
        try:
            if (dic_frame[str(num)]["data"].get("name") != None):
                name_list.append(dic_frame[str(num)]["data"]["name"])
            else:
                continue   
        except:
            continue
        try:
            if (dic_frame[str(num)]["data"].get("detailed_description") != None):
                description_list.append(dic_frame[str(num)]["data"]["detailed_description"])
            else:
                description_list.append(None)
        except:
            continue
        try:
            if (dic_frame[str(num)]["data"].get("price_overview") != None):
                price.append(dic_frame[str(num)]["data"]["price_overview"]["final_formatted"])
            else:
                price.append(None)
        except:
            continue
        try:            
            if (dic_frame[str(num)]["data"].get("genres") != None):
                genr.append(dic_frame[str(num)]["data"]["genres"][0]["description"])
            else:
                genr.append(None)
        except:
            continue
        try:
            if(dic_frame[str(num)]["data"].get("recommendations") != None):
                recom.append(str(dic_frame[str(num)]["data"]["recommendations"])[10:-1])
            else:
                recom.append(None)
        except:
            continue
        try:
            if(dic_frame[str(num)]["data"].get("categories") != None):
                temp_len=len(dic_frame[str(num)]["data"]["categories"])
                for j in range(temp_len):
                    temp_category_list.append(dic_frame[str(num)]["data"]["categories"][j]["description"])
                categor.append(temp_category_list)
            else:
                categor.append(None)
        except:
            continue

In [11]:
#게임별 정보 리스트
name_lst=[]
description_lst=[]
price_lst=[]
categories_lst=[]
genres_lst=[]
recommendations_lst=[]

In [12]:
extract_game_info(dic_frame,app_id_lst,name_lst,description_lst,price_lst,categories_lst,genres_lst,recommendations_lst)

In [13]:
print(len(name_lst))
print(len(description_lst))
print(len(price_lst))
print(len(categories_lst))
print(len(genres_lst))
print(len(recommendations_lst))

2851
2851
2851
2851
2851
2851


In [14]:
app_detail_data=pd.DataFrame(name_lst)

In [15]:
app_detail_data["description"]=description_lst
app_detail_data["price"]=price_lst
app_detail_data["categories"]=categories_lst
app_detail_data["genres"]=genres_lst
app_detail_data["recommendations"]=recommendations_lst

In [16]:
app_detail_data

,0,description,price,categories,genres,recommendations
0,Counter-Strike,Play the world's number 1 online action game. ...,"₩ 10,500","[Multi-player, PvP, Online PvP, Shared/Split S...",Action,98065
1,Team Fortress Classic,One of the most popular online action games of...,"₩ 5,500","[Multi-player, PvP, Online PvP, Shared/Split S...",Action,3664
2,Deathmatch Classic,Enjoy fast-paced multiplayer gaming with Death...,"₩ 5,500","[Multi-player, PvP, Online PvP, Shared/Split S...",Action,1250
3,Half-Life: Opposing Force,Return to the Black Mesa Research Facility as ...,"₩ 5,500","[Single-player, Multi-player, Valve Anti-Cheat...",Action,7834
4,Ricochet,A futuristic action game that challenges your ...,"₩ 5,500","[Multi-player, PvP, Online PvP, Valve Anti-Che...",Action,2637
...,...,...,...,...,...,...
2846,Action Taimanin,"<img src=""https://cdn.cloudflare.steamstatic.c...",None,"[Single-player, Multi-player, MMO, PvP, Online...",Action,None
2847,Mr. Prepper: Prologue,<h1>Also check the game from our friends :)</h...,None,"[Single-player, Steam Achievements, Captions a...",Action,None
2848,Siege Survival: Gloria Victis Prologue,<h1>WISHLIST THE FULL VERSION OF THE GAME</h1>...,None,[Single-player],Adventure,None
2849,Healing Movement,This app works by balancing the elements.<br /...,None,[Single-player],Casual,None


In [21]:
app_detail_data.rename(columns = {0 : 'game name'}, inplace = True)

In [22]:
app_detail_data

,game name,description,price,categories,genres,recommendations
0,Counter-Strike,Play the world's number 1 online action game. ...,"₩ 10,500","[Multi-player, PvP, Online PvP, Shared/Split S...",Action,98065
1,Team Fortress Classic,One of the most popular online action games of...,"₩ 5,500","[Multi-player, PvP, Online PvP, Shared/Split S...",Action,3664
2,Deathmatch Classic,Enjoy fast-paced multiplayer gaming with Death...,"₩ 5,500","[Multi-player, PvP, Online PvP, Shared/Split S...",Action,1250
3,Half-Life: Opposing Force,Return to the Black Mesa Research Facility as ...,"₩ 5,500","[Single-player, Multi-player, Valve Anti-Cheat...",Action,7834
4,Ricochet,A futuristic action game that challenges your ...,"₩ 5,500","[Multi-player, PvP, Online PvP, Valve Anti-Che...",Action,2637
...,...,...,...,...,...,...
2846,Action Taimanin,"<img src=""https://cdn.cloudflare.steamstatic.c...",None,"[Single-player, Multi-player, MMO, PvP, Online...",Action,None
2847,Mr. Prepper: Prologue,<h1>Also check the game from our friends :)</h...,None,"[Single-player, Steam Achievements, Captions a...",Action,None
2848,Siege Survival: Gloria Victis Prologue,<h1>WISHLIST THE FULL VERSION OF THE GAME</h1>...,None,[Single-player],Adventure,None
2849,Healing Movement,This app works by balancing the elements.<br /...,None,[Single-player],Casual,None


In [23]:
app_detail_data.to_csv("./app_detail_data_1.csv")